In [2]:
#Embedding 계층 구현
import numpy as np
W = np.arange(21).reshape(7,3)
W#가중치

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [3]:
W[2]#원하는 행 명시

array([6, 7, 8])

In [4]:
W[5]#원하는 행 명시

array([15, 16, 17])

In [5]:
#가중치 W로부터 여러 행을 한꺼번에 추출
idx = np.array([1,0,3,0])
idx

array([1, 0, 3, 0])

In [6]:
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [7]:
sample_weights = [[1,2],[3,4]]
np.zeros_like(sample_weights)

array([[0, 0],
       [0, 0]])

In [8]:
a, = [1]
a

1

word2vec의 구현의 입력측 MatMul 계층을 Embedding 계층으로 전환

In [9]:
#Embedding 계층의 forward() 메서드를 구현
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx =idx#추출하는 행의 인덱스를 배열로 저장
        out = W[idx]
        return out
    '''
    <비추 backward>
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0#dW의 원소를 0으로 덮어씀
        dW[self.idx] = dout#앞 층에서 전해진 기울기 dout을 idx번째 행에 할당
        return None
    '''
    #올바른 역전파
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        
        #중복 문제 해결하기 위해 '할당'이 아닌 '더하기'를 해야 함
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        #혹은 dout를 dW의 self.idx번째 행에 더해줌(numpy가 더 빠름)
        #np.add.at(dW, self.idx, dout)
        return None

In [10]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)#embedding 계층
        self.params = self.embed.params#매개변수 저장
        self.grads = self.embed.grads#기울기 저장
        self.cache = None#순전파 시의 계산 결과를 잠시 유지하기 위한 변수
    
    def forward(self, h, idx):#순전파
        target_W = self.embed.forward(idx)
        out = np.sum(target_W*h, axis = 1)#내적 계산
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):#역전파
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0],1)
        
        dtarget_W = dout*h
        self.embed.backward(dtarget_W)
        dh = dout*target_W
        return dh